In [1]:
import sys
sys.path.append('C:/Users/Public/Documents/ShalabyGroup/aimsun_ddqn_server - sig/modules/')
import ddqn_framework as ddqn
import mlp_framework as nn
import pickle
import numpy as np

# STEP 1: create configuration
if __name__ == '__main__':
    configuration = ddqn.trainer_config(app_name='AimsunNext',
                                    # BUFFER_SIZE = 50e3,
                                    BUFFER_SIZE = 50000,
                                    # STEPS_PER_EPISODE = 500,
                                    STEPS_PER_EPISODE=500,
                                    MAX_STEPS = 9999999,
                                    # UPDATE_TARGET_STEPS = 1000,
                                    UPDATE_TARGET_STEPS=1000,
                                    BATCH_SIZE = 32,
                                    GAMMA = 0.9,
                                    EXPLORATION = 5000,a
                                    E_MIN = 0.1,
                                    priority = True,
                                    # alpha = 0.001,
                                    alpha = 0.01,
                                    epsilon = 0.1
                                    )

    # STEP 2: build models (online & target)
    A1 = nn.layer(configuration.INPUT_SIZE, nodes=512)
    A2 = nn.layer(512, 512)
    A3 = nn.layer(512, 512)
    A4 = nn.layer(512, 512)
    AOUT = nn.layer(512, configuration.OUTPUT_SIZE)
    AOUT.f = nn.f_iden

    L1 = nn.layer(configuration.INPUT_SIZE, nodes=512)
    L2 = nn.layer(512, 512)
    L3 = nn.layer(512, 512)
    L4 = nn.layer(512, 512)
    LOUT = nn.layer(512, configuration.OUTPUT_SIZE)
    LOUT.f = nn.f_iden
    
    L1.w = np.copy(A1.w)
    L2.w = np.copy(A2.w)
    L3.w = np.copy(A3.w)
    L4.w = np.copy(A4.w)
    LOUT.w = np.copy(AOUT.w)
    
    onlineNet = nn.mlp([A1, A2, A3, A4, AOUT])
    onlineNet.erf = nn.log_cosh  # cost in prev aimsun_dqn
    targetNet = nn.mlp([L1, L2, L3, L4, LOUT])
    targetNet.erf = nn.log_cosh  # cost in prev aimsun_dqn

    # STEP 3: create trainer
    ddqn_model = ddqn.trainer(onlineNet,targetNet,configuration)
    ddqn_model.load_model(False)
    
    # STEP 5: train the trainer (dqn_model) for configuration.MAX_STEPS:
    ddqn_model.train(log=True)